In [1]:
import datetime as dt
import pandas as pd

In [2]:
# identifier for each team on pro-football-reference
teams = ['crd', # cardinals
         'atl', # falcons
         'rav', # ravens
         'buf', # bills
         'car', # panthers
         'chi', # bears
         'cin', # bengals
         'cle', # browns
         'dal', # cowboys
         'den', # denver
         'det', # lions
         'gnb', # packers
         'htx', # texans
         'clt', # colts
         'jax', # jaguars
         'kan', # chiefs
         'sdg', # chargers
         'ram', # rams
         'mia', # dolphins
         'min', # vikings
         'nwe', # patriots
         'nor', # saints
         'nyg', # giants
         'nyj', # jets
         'rai', # raiders
         'phi', # eagles
         'pit', # steelers
         'sfo', # 49ers
         'sea', # seahawks
         'tam', # buccs
         'oti', # titans
         'was' # football team
        ]
len(teams)

32

In [3]:
# year range starts in 2002 because the houston texans didn't exist in a prior form, Causing it to kick an error later in the program.
year_range = range(2002,2020)
year_dict = {}

In [4]:
%%time
# general data ingestion

for year in year_range:
    print(f'Starting: {year}')
    temp_dict={}
    for team in teams:
        url = f'https://www.pro-football-reference.com/teams/{team}/{year}.htm'
        data = pd.read_html(url, header=1)
        temp_dict[team] = data[1]
        temp_dict[team]['year'] = year
        temp_dict[team]['team'] = team
        print(f'          {team} complete')
    year_dict[year] = temp_dict

Starting: 2002
          crd complete
          atl complete
          rav complete
          buf complete
          car complete
          chi complete
          cin complete
          cle complete
          dal complete
          den complete
          det complete
          gnb complete
          htx complete
          clt complete
          jax complete
          kan complete
          sdg complete
          ram complete
          mia complete
          min complete
          nwe complete
          nor complete
          nyg complete
          nyj complete
          rai complete
          phi complete
          pit complete
          sfo complete
          sea complete
          tam complete
          oti complete
          was complete
Starting: 2003
          crd complete
          atl complete
          rav complete
          buf complete
          car complete
          chi complete
          cin complete
          cle complete
          dal complete
          den complete
    

In [5]:
year_dict[2002]['chi'].head()

,Week,Day,Date,Unnamed: 3,Unnamed: 4,Unnamed: 5,OT,Rec,Unnamed: 8,Opp,...,1stD.1,TotYd.1,PassY.1,RushY.1,TO.1,Offense,Defense,Sp. Tms,year,team
0,1,Sun,September 8,1:04PM ET,boxscore,W,NaN,1-0,NaN,Minnesota Vikings,...,19.0,368.0,228.0,140.0,3.0,6.37,-1.20,1.01,2002,chi
1,2,Sun,September 15,1:04PM ET,boxscore,W,NaN,2-0,@,Atlanta Falcons,...,19.0,257.0,135.0,122.0,3.0,-9.67,8.06,3.87,2002,chi
2,3,Sun,September 22,1:05PM ET,boxscore,L,NaN,2-1,NaN,New Orleans Saints,...,18.0,302.0,229.0,73.0,3.0,0.91,-1.64,-10.48,2002,chi
3,4,Sun,September 29,1:04PM ET,boxscore,L,OT,2-2,@,Buffalo Bills,...,26.0,410.0,307.0,103.0,1.0,3.33,-7.73,0.39,2002,chi
4,5,Mon,October 7,9:08PM ET,boxscore,L,NaN,2-3,NaN,Green Bay Packers,...,20.0,457.0,333.0,124.0,1.0,-13.60,-3.38,9.28,2002,chi


In [6]:
# testing that I can access the dataframe
year_dict[2002]['chi']

,Week,Day,Date,Unnamed: 3,Unnamed: 4,Unnamed: 5,OT,Rec,Unnamed: 8,Opp,...,1stD.1,TotYd.1,PassY.1,RushY.1,TO.1,Offense,Defense,Sp. Tms,year,team
0,1,Sun,September 8,1:04PM ET,boxscore,W,NaN,1-0,NaN,Minnesota Vikings,...,19.0,368.0,228.0,140.0,3.0,6.37,-1.20,1.01,2002,chi
1,2,Sun,September 15,1:04PM ET,boxscore,W,NaN,2-0,@,Atlanta Falcons,...,19.0,257.0,135.0,122.0,3.0,-9.67,8.06,3.87,2002,chi
2,3,Sun,September 22,1:05PM ET,boxscore,L,NaN,2-1,NaN,New Orleans Saints,...,18.0,302.0,229.0,73.0,3.0,0.91,-1.64,-10.48,2002,chi
3,4,Sun,September 29,1:04PM ET,boxscore,L,OT,2-2,@,Buffalo Bills,...,26.0,410.0,307.0,103.0,1.0,3.33,-7.73,0.39,2002,chi
4,5,Mon,October 7,9:08PM ET,boxscore,L,NaN,2-3,NaN,Green Bay Packers,...,20.0,457.0,333.0,124.0,1.0,-13.60,-3.38,9.28,2002,chi
5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bye Week,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002,chi
6,7,Sun,October 20,1:03PM ET,boxscore,L,OT,2-4,@,Detroit Lions,...,23.0,362.0,170.0,192.0,1.0,-16.58,-1.40,16.23,2002,chi
7,8,Sun,October 27,1:00PM ET,boxscore,L,NaN,2-5,@,Minnesota Vikings,...,24.0,364.0,216.0,148.0,1.0,-16.75,-6.35,0.00,2002,chi
8,9,Sun,November 3,1:05PM ET,boxscore,L,NaN,2-6,NaN,Philadelphia Eagles,...,23.0,309.0,192.0,117.0,1.0,-8.98,1.96,0.70,2002,chi
9,10,Sun,November 10,4:15PM ET,boxscore,L,NaN,2-7,NaN,New England Patriots,...,19.0,397.0,315.0,82.0,2.0,-5.48,0.32,-0.61,2002,chi


In [7]:
# dataframe cleaning
column_clean_dict =  {'Unnamed: 3':'Kickoff_time',
     'Unnamed: 5': 'W/L',
     'Unnamed: 8' : 'Home/Away',
     'Tm' : 'Team_Score',
     'Opp.1' : 'Oppo_score',
     '1stD' : '1D_gained',
     'TotYd' : 'Team_Total_Yards',
     'PassY' : 'Team_Pass_Yards',
     'RushY' : 'Team_Rush_Yards',
     'TO' : 'team_turnover',
     '1stD.1' : '1D_allowed',
     'TotYd.1' : 'oppo_total_yards',
     'PassY.1' : 'oppo_pass_yards',
     'RushY.1' : 'oppo_rush_yards',
     'TO.1' : 'oppo_turnover',
     'Offense' : 'offense_expected_points',
     'Defense' : 'defense_expected_points',
     'Sp. Tms' : 'sp_expected_points'
    }

In [11]:
# cleans each dataframe
for year in year_dict:
    for team in year_dict[year]:
        year_dict[year][team].rename(column_clean_dict, inplace=True, axis=1)
        year_dict[year][team].drop('Unnamed: 4', axis=1, inplace=True)
year_dict[2002]['sea'].head()

,Week,Day,Date,Kickoff_time,W/L,OT,Rec,Home/Away,Opp,Team_Score,...,1D_allowed,oppo_total_yards,oppo_pass_yards,oppo_rush_yards,oppo_turnover,offense_expected_points,defense_expected_points,sp_expected_points,year,team
0,1,Sun,September 8,4:17PM ET,L,NaN,0-1,@,Oakland Raiders,17.0,...,27.0,423.0,202.0,221.0,2.0,-1.72,-15.40,-0.77,2002,sea
1,2,Sun,September 15,4:05PM ET,L,NaN,0-2,NaN,Arizona Cardinals,13.0,...,19.0,347.0,98.0,249.0,1.0,-6.43,-3.29,1.44,2002,sea
2,3,Sun,September 22,4:14PM ET,L,NaN,0-3,@,New York Giants,6.0,...,21.0,351.0,257.0,94.0,NaN,-13.27,2.43,5.93,2002,sea
3,4,Sun,September 29,8:35PM ET,W,NaN,1-3,NaN,Minnesota Vikings,48.0,...,28.0,443.0,249.0,194.0,4.0,7.97,5.59,8.90,2002,sea
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bye Week,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002,sea
